In [14]:
import numpy as np
import open3d as o3d

def create_coordinate_frame(size=100.0):
    """创建坐标系模型"""
    return o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)

# 读取点云
# pcd = o3d.io.read_point_cloud("./pointcloud/transformed_pcd.pcd")
pcd = o3d.io.read_point_cloud("./pointcloud/init.ply")
points = np.asarray(pcd.points)

# 获取点云范围
min_bound = points.min(axis=0)  # 最小xyz值
max_bound = points.max(axis=0)  # 最大xyz值
center = (min_bound + max_bound) / 2  # 点云中心

print("点云范围：")
print(f"最小边界: {min_bound}")
print(f"最大边界: {max_bound}")
print(f"点云中心: {center}")
# 创建坐标系
coordinate_frame = create_coordinate_frame(size=50.0)

# 将坐标系平移到点云中心
# coordinate_frame.translate(max_bound)

# 可视化点云和坐标系
o3d.visualization.draw_geometries([pcd, coordinate_frame],
                                  window_name="Point Cloud with Coordinate Frame",
                                  width=800,
                                  height=600,
                                  left=50,
                                  top=50,
                                  point_show_normal=False,
                                  mesh_show_wireframe=False,
                                  mesh_show_back_face=False)

点云范围：
最小边界: [  1.86000001   0.         -28.46221924]
最大边界: [ 93.48000336 984.40002441  19.57814026]
点云中心: [ 47.67000169 492.20001221  -4.44203949]


In [9]:
import numpy as np
import open3d as o3d
def get_height_color(points):
    # 获取z轴高度
    z = points[:, 2]
    
    # 归一化到0-1范围
    z_min, z_max = np.min(z), np.max(z)
    z_normalized = (z - z_min) / (z_max - z_min)
    
    # 创建颜色数组
    colors = np.zeros((len(points), 3))
    
    # 使用jet颜色映射
    colors[:, 0] = np.clip(1.5 - abs(2.0 - 4.0 * z_normalized), 0, 1)  # R
    colors[:, 1] = np.clip(1.5 - abs(2.0 - 4.0 * (z_normalized - 0.5)), 0, 1)  # G
    colors[:, 2] = np.clip(1.5 - abs(2.0 - 4.0 * (z_normalized - 1.0)), 0, 1)  # B
    
    return colors

# 读取点云
# workpiece = 'thickBoardweld.pcd'
workpiece = 'test_zhongjian.pcd'
pcd = o3d.io.read_point_cloud("./transforms/" + workpiece + "/transformed_pcd.pcd")
# original_points = np.asarray(pcd.points)

# 创建 original_points 的副本
# original_points = np.array(pcd.points)

# 获取点云中的点
points = np.asarray(pcd.points)
# 创建 original_points 的副本
original_points = points.copy()

# 为点云设置颜色
colors = get_height_color(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# 创建坐标系
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50.0,
    origin=[0, 0, 0]
)

# 创建可视化器
vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window()

# 添加点云和坐标系到可视化器
vis.add_geometry(pcd)
vis.add_geometry(mesh_frame)

# 设置初始视角
opt = vis.get_render_option()
opt.background_color = np.asarray([1, 1, 1])  # 白色背景
opt.point_size = 1.0


# # 设置初始视角（可选）
# opt = vis.get_render_option()
# opt.background_color = np.asarray([255, 255, 255])  # 设置背景颜色为白色
# opt.point_size = 1.0  # 设置点的大小

# 设置全局变量
x_min = -20
x_max = 200
step = 10

def update_point_cloud():
    # points = original_points
    global pcd
    points = original_points.copy()
    
    print(f"original_points点云X范围: {np.min(points[:, 0])} 到 {np.max(points[:, 0])}")
    mask = (points[:, 0] >= x_min) & (points[:, 0] <= x_max)
    filtered_points = points[mask]
    xMax = np.max(filtered_points[:, 0])
    xMin = np.min(filtered_points[:, 0])
    print(f"filtered_points点云X范围: {xMin} 到 {xMax}")
    pcd.points = o3d.utility.Vector3dVector(filtered_points)

    # 如果使用了颜色，也需要更新颜色
    # if hasattr(pcd, 'colors'):
    colors = np.zeros((len(points), 3)) + [1, 0, 0]  # 全红色
    # colors = get_height_color(filtered_points)  # 或其他颜色生成方法
    pcd.colors = o3d.utility.Vector3dVector(colors)

    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()
    print(f"X范围: {x_min} 到 {x_max}")

# 定义按键回调函数
def change_x_min_minus(vis):
    global x_min
    x_min -= step
    update_point_cloud()
    return False

def change_x_min_plus(vis):
    global x_min
    x_min += step
    update_point_cloud()
    return False

def change_x_max_minus(vis):
    global x_max
    x_max -= step
    update_point_cloud()
    return False

def change_x_max_plus(vis):
    global x_max
    x_max += step
    update_point_cloud()
    return False

# 注册按键回调
vis.register_key_callback(ord('A'), change_x_min_minus)  # A键减小x_min
vis.register_key_callback(ord('D'), change_x_min_plus)   # D键增加x_min
vis.register_key_callback(ord('J'), change_x_max_minus)  # J键减小x_max
vis.register_key_callback(ord('L'), change_x_max_plus)   # L键增加x_max

# 打印使用说明
print("按键控制说明:")
print("A/D: 减小/增加最小X值")
print("J/L: 减小/增加最大X值")

# 更新初始显示
update_point_cloud()

# 运行可视化
vis.run()
vis.destroy_window()

[Open3D WARNING] Read PCD failed: unable to open file: ./transforms/./pointcloud/init.ply/transformed_pcd.pcd


ValueError: zero-size array to reduction operation minimum which has no identity

In [10]:
import numpy as np
import open3d as o3d

# 读取点云数据
pcd = o3d.io.read_point_cloud("./pointcloud/transformed_pcd.pcd")

# 获取点云中的点
points = np.asarray(pcd.points)

# 创建 original_points 的副本
original_points = points.copy()

# 定义切除函数
def update_point_cloud(x_min, x_max):
    # global original_points
    points = original_points.copy()  # 使用副本进行操作
    mask = (points[:, 0] >= x_min) & (points[:, 0] <= x_max)
    filtered_points = points[mask]
    
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

    # # pcd.points = o3d.utility.Vector3dVector(filtered_points)

    # vis.update_geometry(filtered_pcd)  # 只能更新已经添加到可视化器中的几何体
    # vis.poll_events()
    # vis.update_renderer()
    
    vis.clear_geometries()
    vis.add_geometry(filtered_pcd)
    vis.add_geometry(mesh_frame)
    vis.poll_events()
    vis.update_renderer()

# 创建可视化器
vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window()

# 添加点云和坐标系到可视化器
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=50.0, origin=[0, 0, 0])
vis.add_geometry(pcd)
vis.add_geometry(mesh_frame)

# 设置初始视角（可选）
opt = vis.get_render_option()
opt.background_color = np.asarray([255, 255, 255])  # 设置背景颜色为白色
opt.point_size = 1.0  # 设置点的大小

# 定义按键回调函数
def change_x_min_minus(vis):
    global x_min
    x_min -= step
    update_point_cloud(x_min, x_max)
    return False

def change_x_min_plus(vis):
    global x_min
    x_min += step
    update_point_cloud(x_min, x_max)
    return False

def change_x_max_minus(vis):
    global x_max
    x_max -= step
    update_point_cloud(x_min, x_max)
    return False

def change_x_max_plus(vis):
    global x_max
    x_max += step
    update_point_cloud(x_min, x_max)
    return False

# 注册按键回调
vis.register_key_callback(ord('A'), change_x_min_minus)  # A键减小x_min
vis.register_key_callback(ord('D'), change_x_min_plus)   # D键增加x_min
vis.register_key_callback(ord('J'), change_x_max_minus)  # J键减小x_max
vis.register_key_callback(ord('L'), change_x_max_plus)   # L键增加x_max

# 打印使用说明
print("按键控制说明:")
print("A/D: 减小/增加最小X值")
print("J/L: 减小/增加最大X值")

# 初始化参数
x_min = -20
x_max = 180
step = 5

# 更新初始显示
update_point_cloud(x_min, x_max)

# 运行可视化
vis.run()
vis.destroy_window()

按键控制说明:
A/D: 减小/增加最小X值
J/L: 减小/增加最大X值


In [8]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
print(a.shape)
mask = a[:, 0] > 2
print(mask)
print(a[mask])

(4, 3)
[False  True  True  True]
[[ 4  5  6]
 [ 7  8  9]
 [10 11 12]]
